# Time Series Prediction with Reservoir Computing ($RCN$)

AUTHOR: Anurag Dutta (anuragdutta.research@gmail.com)

SUPERVISOR: Tanujit Chakraborty (tanujit.chakraborty@sorbonne.ae)

CREATED: 2023-03-08 17:24:53

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Gathering Dependencies

_Importing Required Libraries_

In [2]:
pip install hampel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
import keras
import tensorflow as tf
from hampel import hampel
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Loading Datasets

_दिल्ली WIND SPEED_

In [4]:
data = pd.read_csv("/content/drive/MyDrive/new/datasets/delhi.csv")
training_set = data.iloc[:, 3]
print(training_set)

0       0.000000
1       2.980000
2       4.633333
3       1.233333
4       3.700000
          ...   
1457    3.547826
1458    6.000000
1459    6.266667
1460    7.325000
1461    0.000000
Name: wind_speed, Length: 1462, dtype: float64


## Preprocessing the Data

In [5]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(training_set, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[0.         2.98       4.63333333] 1.2333333333333334
[2.98       4.63333333 1.23333333] 3.7
[4.63333333 1.23333333 3.7       ] 1.48
[1.23333333 3.7        1.48      ] 6.3
[3.7  1.48 6.3 ] 7.142857142857143
[1.48       6.3        7.14285714] 12.5
[ 6.3         7.14285714 12.5       ] 7.399999999999999
[ 7.14285714 12.5         7.4       ] 10.571428571428571
[12.5         7.4        10.57142857] 13.228571428571428
[ 7.4        10.57142857 13.22857143] 4.633333333333334
[10.57142857 13.22857143  4.63333333] 0.6166666666666667
[13.22857143  4.63333333  0.61666667] 0.5285714285714286
[4.63333333 0.61666667 0.52857143] 0.0
[0.61666667 0.52857143 0.        ] 5.250000000000001
[0.52857143 0.         5.25      ] 8.950000000000001
[0.   5.25 8.95] 5.883333333333333
[5.25       8.95       5.88333333] 8.471428571428572
[8.95       5.88333333 8.47142857] 2.22
[5.88333333 8.47142857 2.22      ] 3.083333333333333
[8.47142857 2.22       3.08333333] 4.016666666666667
[2.22       3.08333333 4.01666667]

## Model ($RCN$)

In [6]:
pip install reservoirpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from reservoirpy.nodes import Reservoir, Ridge

reservoir = Reservoir(units=100, lr=0.5, sr=0.001)
readout = Ridge(output_dim=3, ridge=1e-8)

In [8]:
esn = reservoir >> readout

In [9]:
esn.fit(X[:292], X[1:293], warmup=100)

Running Model-0:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-0: 292it [00:00, 3853.24it/s]           
Running Model-0: 100%|██████████| 1/1 [00:00<00:00, 10.67it/s]


Fitting node Ridge-0...


'Model-0': Model('Reservoir-0', 'Ridge-0')

In [10]:
predictions = esn.run(X[293:-1])

Running Model-0: 1165it [00:00, 5333.36it/s]


## Metrics ($RCN$)

In [11]:
import reservoirpy.observables
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(X[293:-1], predictions))
mae = np.sqrt(mean_absolute_error(X[293:-1], predictions))
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

Test RMSE: 4.469
Test MAE: 1.840
